In [ ]:
#Pattern 1

In [4]:
import pandas as pd
import glob

from PyPDF2 import PdfReader

In [16]:
from pdfminer.layout import LAParams, LTTextBox
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.converter import TextConverter
import io

resource_manager = PDFResourceManager()
fake_file_handle = io.StringIO()
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
page_interpreter = PDFPageInterpreter(resource_manager, converter)


def pdf_get_text(fn):
    
    from pdfminer.layout import LAParams, LTTextBox
    from pdfminer.pdfpage import PDFPage
    from pdfminer.pdfinterp import PDFResourceManager
    from pdfminer.pdfinterp import PDFPageInterpreter
    from pdfminer.converter import PDFPageAggregator
    from pdfminer.converter import TextConverter
    import io

    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    

    with open(fn, 'rb') as fh:

        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    close open handles
    converter.close()
    fake_file_handle.close()

    return text

In [18]:
def pdf_to_csv(filename):
    from cStringIO import StringIO  
    from pdfminer.converter import LTChar, TextConverter    #<-- changed
    from pdfminer.layout import LAParams
    from pdfminer.pdfparser import PDFDocument, PDFParser
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

    class CsvConverter(TextConverter):
        def __init__(self, *args, **kwargs):
            TextConverter.__init__(self, *args, **kwargs)

        def end_page(self, i):
            from collections import defaultdict
            lines = defaultdict(lambda : {})
            for child in self.cur_item.objs:
                if isinstance(child, LTChar):               #<-- changed
                    (_,_,x,y) = child.bbox                   
                    line = lines[int(-y)]
                    line[x] = child.text.encode(self.codec)

            for y in sorted(lines.keys()):
                line = lines[y]
                self.outfp.write(";".join(line[x] for x in sorted(line.keys())))
                self.outfp.write("\n")

    # ... the following part of the code is a remix of the 
    # convert() function in the pdfminer/tools/pdf2text module
    rsrc = PDFResourceManager()
    outfp = StringIO()
    device = CsvConverter(rsrc, outfp, codec="utf-8", laparams=LAParams())  #<-- changed
        # becuase my test documents are utf-8 (note: utf-8 is the default codec)

    doc = PDFDocument()
    fp = open(filename, 'rb')
    parser = PDFParser(fp)       
    parser.set_document(doc)     
    doc.set_parser(parser)       
    doc.initialize('')

    interpreter = PDFPageInterpreter(rsrc, device)

    for i, page in enumerate(doc.get_pages()):
        outfp.write("START PAGE %d\n" % i)
        if page is not None:
            interpreter.process_page(page)
        outfp.write("END PAGE %d\n" % i)

    device.close()
    fp.close()

    return outfp.getvalue()

In [17]:
for fn in glob.glob('all_collections/*.pdf'):

    print(fn)        
    
    text = pdf_get_text(fn)

    idx1 = text.find('REFERENCES')
    idx2 = text.find('References')
    
    if idx1 != -1:
        print ('Found: REFERENCES', idx1)
    elif idx2 != -1:
        print ('Found: References', idx2)
    else:
        print('??????????')

all_collections/Using image recognition for trading.pdf
??????????
all_collections/An Automatic Stock Trading System using Particle Swarm Optimization .pdf
Found: REFERENCES 80705
all_collections/thru_earnings_ws_estimize_delta_event_study.pdf
Found: REFERENCES 80705
all_collections/Best technical indicators for Bitcoin from TA-lib _ by Berend _ Coinmonks _ Mar, 2022 _ Medium.pdf
Found: REFERENCES 80705
all_collections/Deep learning for relaive stock performance prediction.pdf
Found: REFERENCES 80705
all_collections/Stock Prices Prediction using Deep Learning Models.pdf


error: unpack requires a buffer of 2 bytes

In [65]:
skipped = ['all_collections/Fuzzy Based Modified SHL algorithm for Spiking Neural  Network.pdf', 
           'all_collections/A Survey on Unsupervised Machine Learning Algorithms for Automation, Classification and Maintenance.pdf',
          'all_collections/Evolutionary Function Approximation for Reinforcement Learning.pdf',
          'all_collections/Purified sentiment indicators for the stock market5.04.09.pdf',
           'all_collections/Time Series Analysis by State Space Methods (2ed).pdf',
           'all_collections/Machine Learning for Time Series Anomaly Detection.pdf',
           'all_collections/AI-and-Financial-Markets.pdf',
           'all_collections/normalize-time-series-and-forecast-using-evolutionary-neural-network.pdf',
           'all_collections/Rough Sets.pdf',
           'all_collections/Deep reinforcement learning stock market trading, utilizing a CNN with candlestick images.pdf',
            'all_collections/Genetic Algorithm- An Application to Technical Trading System Design.pdf',
            'all_collections/GA_an_application_to_technical_trading_system_design.pdf',
           'all_collections/Action-specialized expert ensemble trading system with extended discrete action space using deep reinforcement learning.pdf',
           'all_collections/Deep reinforcement learning stock market trading, utilizing a CNN with candlestick.pdf',
           'all_collections/Multi-DQN: An ensemble of Deep Q-learning agents for stock  market forecasting.pdf',
           'all_collections/Monte-Carlo Evaluation of Trading Systems.pdf',
           'all_collections/New developments in time series econometrics: An overview.pdf',
           'all_collections/Search and Reasoning in Problem Solving.pdf',
           'all_collections/DPP: Deep predictor for price movement from candlestick charts.pdf',
           'all_collections/Building Reliable Trading Systems.pdf',
           'all_collections/Effects of Time Normalization on the Accuracy of the Success of Artificial Neural Network Model.pdf',  
           'all_collections/Impact of chart image characteristics on stock price prediction with a convolutional neural network.pdf',
           'all_collections/A Comprehensive Survey of Time Series Anomaly Detection in Online Social Network Data.pdf',
           'all_collections/A Comprehensive Survey of Time Series Anomaly Detection in Online Social Network Data.pdf',
           'all_collections/Selection of the optimal trading model for stock investment in different industries.pdf',
           'all_collections/The Effect of the Normalization Method Used in Different Sample Sizes on.pdf',
           'all_collections/Genetic Algorithm: An Application to Technical Trading System Design.pdf',
          'all_collections/Improving stock trading decisions based on pattern recognition using machine learning technology.pdf']


cnt = 0

tot_ref = ''

for fn in glob.glob('all_collections/*.pdf'):
        
    if fn in skipped:
        continue
    
    if cnt >= 0:
        reader = PdfReader(fn)

        text = ""

        for page in reader.pages:
            text += page.extract_text() + "\n"

        idx1 = text.find('REFERENCES')
        idx2 = text.find('References')

        if idx1 != -1:
            print ('Found: REFERENCES', idx1)
            tot_ref += text[idx1:]
        elif idx2 != -1:
            print ('Found: References', idx2)
            tot_ref += text[idx2:]
        else:
            print('??????????')

        tot_ref += '\n\n ********************\n\n'
        print(cnt, fn)
    
    cnt += 1

??????????
0 all_collections/Using image recognition for trading.pdf
??????????
1 all_collections/An Automatic Stock Trading System using Particle Swarm Optimization .pdf
??????????
2 all_collections/thru_earnings_ws_estimize_delta_event_study.pdf
??????????
3 all_collections/Best technical indicators for Bitcoin from TA-lib _ by Berend _ Coinmonks _ Mar, 2022 _ Medium.pdf
??????????
4 all_collections/Deep learning for relaive stock performance prediction.pdf
Found: References 29093
5 all_collections/Stock Prices Prediction using Deep Learning Models.pdf
??????????
6 all_collections/Attentive Neural Models for Algorithmic Trading-1.pdf
Found: References 84862
7 all_collections/Optimization Techniques: An Overview.pdf
Found: References 56566
8 all_collections/A multi-layer and multi-ensemble stock trader using deep learning and deep reinforcement learning .pdf
Found: REFERENCES 187895
9 all_collections/Deep Reinforcement Learning- An Overview.pdf
Found: REFERENCES 31650
10 all_collectio

??????????
75 all_collections/Data mining for imbalanced data:Improving classifiers by selective pre-processing of examples .pdf
??????????
76 all_collections/Reinforcement learning improves behaviour from evaluative feedback.pdf
??????????
77 all_collections/Adaptive Normalization: A Novel Data Normalization Approach for Non-Stationary Time Series .pdf
Found: References 30751
78 all_collections/C__Users_matqsb_Documents_My files_Research_Presentation_Mike Paper 3_Salah_post_thesis Oct 20 v3.dvi.pdf
Found: References 55865
79 all_collections/Volume Weighted Average Price Optimal Execution.pdf
??????????
80 all_collections/Dynamics of Gene Expression in an Artificial Genome.pdf
Found: REFERENCES 15561
81 all_collections/A-Review-On-Big-Data-With-Machine-Learning-And-Fuzzy-Logic-For-Better-Decision-Making.pdf
Found: REFERENCES 24462
82 all_collections/Reinforcement Learning Algorithms for Automated Stock Trading.pdf
Found: References 184221
83 all_collections/technical-analysis-of-stocks

Found: REFERENCES 248503
147 all_collections/A Survey on Policy Search for Robotics.pdf
Found: REFERENCES 68918
148 all_collections/A Survey on Transfer Learning.pdf
??????????
149 all_collections/A Deep Reinforcement Learning Framework for the Financial Portfolio Management Problem.pdf
??????????
150 all_collections/Time-Series-Forecasting-of-Amazon-Stock-Prices-using-Neural-Networks-LSTM-and-GAN.pdf
Found: References 22637
151 all_collections/Feature Selection Based on Evolutionary Algorithms for Affective Computing and Stress Recognition.pdf
??????????
152 all_collections/(paper) Transformers in Time Series ; A Survey - AAA (All About AI).pdf
Found: REFERENCES 23420
153 all_collections/A REVIEW PAPER ON PORTFOLIO OPTIMIZATION USING GENETIC ALGORITHM.pdf
??????????
154 all_collections/Evaluation and potential improvements of a deep reinforcement learning model for automated stock trading.pdf
??????????
155 all_collections/Fidelity Learning Center_ Technical Analysis Indicator Guide -

??????????
219 all_collections/Pairs Rotation With Ehlers Loops.pdf
Found: REFERENCES 54561
220 all_collections/AdaRNN: Adaptive Learning and Forecasting for Time Series∗.pdf
Found: References 139643
221 all_collections/AutoML performance in model fitting .pdf
Found: REFERENCES 8740
222 all_collections/STRUCTURED LOW-RANK MATRIX RECOVERY VIA OPTIMIZATION METHODS.pdf
Found: References 222516
223 all_collections/Language models are few-shot learners.pdf
Found: REFERENCES 59791
224 all_collections/Reinforcement Learning: A Tutorial Survey and Recent Advances.pdf
Found: References 42249
225 all_collections/A Biologically Plausible Supervised Learning Method for Spiking Neural Networks Using the Symmetric STDP Rule.pdf
??????????
226 all_collections/METRIC-SPACE ANALYSIS OF SPIKE TRAINS: THEORY, ALGORITHMS, AND APPLICATION.pdf
Found: REFERENCES 74867
227 all_collections/A Brief Survey of Deep Reinforcement Learning.pdf
??????????
228 all_collections/Technical Analysis Of The Financial Marke

??????????
291 all_collections/A Comparison of Genotype Representations to Acquire Stock Trading Strategy Using Genetic Algorithms.pdf


...

Found: REFERENCES 59948
1202 all_collections/A deep learning architecture for temporal sleep stage classification using multivariate and multimodal time series.pdf
Found: REFERENCES 24132
1203 all_collections/Predicting future trends in stock market by decision tree rough-set based hybrid system with HHMM .pdf
Found: References 28270
1204 all_collections/Testing different Reinforcement Learning configurations for financial trading_ Introduction and applications.pdf
Found: References 26057
1205 all_collections/Challenges and Approaches to Time series forecasting: A Survey.pdf
Found: References 3622
1206 all_collections/Spiking Neural Networks Learning, Applications, and Analysis.pdf
??????????
1207 all_collections/Stock Market Prices Do Not Follow Random Walks.pdf
Found: REFERENCES 62214
1208 all_collections/IFROWANN Imbalanced Fuzzy-Rough Ordered Weighted Average Nearest Neighbor Classification.pdf
Found: REFERENCES 31247
1209 all_collections/The Advance-Decline, New-High, New-Low Mark

Found: References 122779
1273 all_collections/Fundamental Analysis and the Cross-Section of Stock Returns: A Data-Mining Approach.pdf
Found: References 10258
1274 all_collections/Financial Data Modeling by Using Asynchronous Parallel Evolutionary Algorithms.pdf
Found: References 15409
1275 all_collections/Long Short Term Memory Networks for Anomaly Detection in Time Series.pdf
Found: References 17707
1276 all_collections/Optimization of Association Rule Mining using Improved  Genetic Algorithms .pdf
Found: REFERENCES 16251
1277 all_collections/Stock Price Prediction using Reinforcement Learning and Feature Extraction.pdf
Found: REFERENCES 18989
1278 all_collections/Effects of Time Normalization on the Accuracy of Dynamic Time Warping.pdf
??????????
1279 all_collections/Evolving Artificial Neural Networks through Complexification.pdf
??????????
1280 all_collections/THE DOW THEORY AND THE MANAGEMENT OF INVESTMENTS.pdf
Found: References 49510
1281 all_collections/Mitigating Metaphors: A C

Found: References 36344
1347 all_collections/Financial Time Series Prediction using Spiking Neural Networks .pdf
??????????
1348 all_collections/Fuzzy Logic Tutorial_ What is, Architecture, Application, Example.pdf
Found: References 35826
1349 all_collections/Automated trading systems statistical andmachine learning methods and hardware implementation: a survey.pdf
Found: References 12697
1350 all_collections/Evaluation of Fitness Functions for Evolved Stock Market Forecasting.pdf
Found: References 49311
1351 all_collections/Reinforcement_Learning_for_Systematic_FX Trading.pdf
Found: REFERENCES 20578
1352 all_collections/Rough set with Effective Clustering Method.pdf
??????????
1353 all_collections/Predicting Tesla Stocks (TSLA) using Python & PyCaret _ by Muneeb Ahmad _ Dev Genius.pdf
Found: REFERENCES 16390
1354 all_collections/TURNOVER PREDICTION OF SHARES USING DATA MINING TECHNIQUES: A CASE STUDY.pdf
??????????
1355 all_collections/Parallel Evolutionary Algorithms.pdf
??????????
1

??????????
1421 all_collections/Time Series Simulation by Conditional Generative Adversarial Net.pdf
Found: References 19237
1422 all_collections/einforcement-learning-for-trading-Paper.pdf
Found: References 81411
1423 all_collections/A Survey of Forex and Stock Price Prediction Using Deep Learning.pdf
Found: References 29374
1424 all_collections/New Interpretations of Normalization Methods in Deep Learning.pdf
Found: References 13324
1425 all_collections/Alexandros Iosifidis - Time-series data analysis.pdf
??????????
1426 all_collections/The "Liquid Computer": A Nevel Strategy for Real-Time Computing on Time Series.pdf
Found: REFERENCES 18132
1427 all_collections/Discovering Patterns in Sentimental Analysis.pdf
Found: References 30335
1428 all_collections/Computational Finance.pdf
Found: REFERENCES 30860
1429 all_collections/Implementation of a Type-2 Fuzzy Logic Based Prediction System for the Nigerian Stock Exchange.pdf
Found: REFERENCES 23203
1430 all_collections/Stock Market Predi

??????????
1492 all_collections/Time Forecast with TPOT. a Python Automated Machine Learning… _ by Susan Li _ Towards Data Science.pdf
Found: References 32483
1493 all_collections/Convolutional Nonlinear Neighbourhood Components Analysis for Time Series Classification.pdf
Found: References 36909
1494 all_collections/On Using Genetic Algorithms to Search Program Spaces.pdf
Found: References 33572
1495 all_collections/Transformers in Time Series- A Survey.pdf
Found: References 19935
1496 all_collections/Optimal Ratio for Data Splitting.pdf
Found: References 39571
1497 all_collections/Forecasting Volatility of Stock Index- Deep Learning Model with Likelihood-Based Loss Function.pdf
??????????
1498 all_collections/DeMark -- technical analysis_ TD Waldo Patterns _ LiteFinance (ex. LiteForex).pdf
Found: References 33578
1499 all_collections/Hierarchical Adaptive Temporal-Relational Modeling for Stock Trend Prediction.pdf
Found: References 37773
1500 all_collections/Absolute exponential stabi

In [66]:
f = open("Reference_Org.txt", "a")
f.write(tot_ref)
f.close()

In [46]:
tot_ref

'REFERENCES\n[1] T. Yigitcanlar, L. Butler, E. Windle, K. C. Desouza, R. Mehmood,\nand J. M. Corchado, “Can Building “Artificially Intelligent Cities”\nSafeguard Humanity from Natural Disasters, Pandemics, and Other\nCatastrophes? An Urban Scholar’s Perspective,” Sensors, vol. 20, no. 10,\np. 2988, may 2020. [Online]. Available: https://www.mdpi.com/1424-\n8220/20/10/2988\n[2] T. Yigitcanlar, N. Kankanamge, M. Regona, A. Maldonado, B. Rowan,\nA. Ryu, K. C. Desouza, J. M. Corchado, R. Mehmood, and R. Y. M.\nLi, “Artificial Intelligence Technologies and Related Urban Planning\nand Development Concepts: How Are They Perceived and Utilized\nin Australia?”Journal of Open Innovation: Technology, Market, and\nComplexity, vol. 6, no. 4, p. 187, dec 2020. [Online]. Available:\nhttps://www.mdpi.com/2199-8531/6/4/187\n[3] E. Alomari, I. Katib, A. Albeshri, and R. Mehmood, “COVID-19:\nDetecting Government Pandemic Measures and Public Concerns from\nTwitter Arabic Data Using Distributed Machine Lea